<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/dividendoption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities from C++ test
def flat_rate_py(evaluation_date_or_quote, forward_rate_or_dc, day_counter_or_none=None):
    # Overload for (date, quote_handle, dc) and (quote_handle, dc)
    if isinstance(evaluation_date_or_quote, ql.Date): # (date, quote, dc)
        evaluation_date = evaluation_date_or_quote
        forward_rate_obj = forward_rate_or_dc
        day_counter = day_counter_or_none
    else: # (quote, dc)
        evaluation_date = ql.Settings.instance().evaluationDate # Assume today if not given
        forward_rate_obj = evaluation_date_or_quote
        day_counter = forward_rate_or_dc

    if isinstance(forward_rate_obj, ql.Quote):
        quote_handle = ql.QuoteHandle(forward_rate_obj)
    elif isinstance(forward_rate_obj, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(forward_rate_obj))
    else: # Assuming it's already a Handle<Quote>
        quote_handle = forward_rate_obj
    return ql.FlatForward(evaluation_date, quote_handle, day_counter)

def flat_vol_py(evaluation_date_or_quote, vol_level_or_dc, day_counter_or_none=None):
    if isinstance(evaluation_date_or_quote, ql.Date):
        evaluation_date = evaluation_date_or_quote
        vol_level_obj = vol_level_or_dc
        day_counter = day_counter_or_none
    else:
        evaluation_date = ql.Settings.instance().evaluationDate
        vol_level_obj = evaluation_date_or_quote
        day_counter = vol_level_or_dc

    if isinstance(vol_level_obj, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_level_obj)
    elif isinstance(vol_level_obj, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_level_obj))
    else:
        vol_quote_handle = vol_level_obj
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), vol_quote_handle, day_counter)

def exercise_type_to_string_py(exercise):
    if isinstance(exercise, ql.EuropeanExercise): return "European"
    if isinstance(exercise, ql.AmericanExercise): return "American"
    return "UnknownExercise"

def payoff_type_to_string_py(payoff):
    if isinstance(payoff, ql.PlainVanillaPayoff): return "PlainVanilla"
    return "UnknownPayoff"

def relative_error_py(x1, x2, reference):
    if reference != 0.0:
        return abs(x1 - x2) / reference
    else: # Fallback if reference is zero
        return abs(x1 - x2)

class DividendOptionTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Most tests set their own eval date, but a default for setup
        self.today = ql.Date(15, ql.May, 2007) # Arbitrary, consistent date
        ql.Settings.instance().evaluationDate = self.today

        self.dc = ql.Actual360()
        self.spot_q = ql.SimpleQuote(0.0)
        self.q_rate_q = ql.SimpleQuote(0.0)
        self.r_rate_q = ql.SimpleQuote(0.0)
        self.vol_q = ql.SimpleQuote(0.0)

        self.spot_h = ql.QuoteHandle(self.spot_q)
        # Handles to TermStructures are important for process
        self.qTS_h = ql.YieldTermStructureHandle()
        self.rTS_h = ql.YieldTermStructureHandle()
        self.volTS_h = ql.BlackVolTermStructureHandle()

        # Link them in _update_market_data or here with initial flat curves
        self.qTS_h.linkTo(flat_rate_py(self.spot_q.value(), self.q_rate_q, self.dc)) # Placeholder eval date for now
        self.rTS_h.linkTo(flat_rate_py(self.spot_q.value(), self.r_rate_q, self.dc))
        self.volTS_h.linkTo(flat_vol_py(self.spot_q.value(), self.vol_q, self.dc))

        self.process = ql.BlackScholesMertonProcess(self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h)

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def _update_market_data(self, current_eval_date, s, q, r, v):
        self.spot_q.setValue(s)
        self.q_rate_q.setValue(q)
        self.r_rate_q.setValue(r)
        self.vol_q.setValue(v)

        # Re-link term structures with the current evaluation date
        self.qTS_h.linkTo(flat_rate_py(current_eval_date, self.q_rate_q, self.dc))
        self.rTS_h.linkTo(flat_rate_py(current_eval_date, self.r_rate_q, self.dc))
        self.volTS_h.linkTo(flat_vol_py(current_eval_date, self.vol_q, self.dc))
        # The process uses handles, so it will pick up the changes.

    def _report_failure(self, greek_name, payoff, exercise, s, q, r, today,
                        v, expected, calculated, error, tolerance):
        # Ensure rates and vols are floats for formatting
        q_float = q.value() if isinstance(q, ql.Quote) else q
        r_float = r.value() if isinstance(r, ql.Quote) else r
        v_float = v.value() if isinstance(v, ql.Quote) else v

        msg = (f"{exercise_type_to_string_py(exercise)} "
               f"{payoff.optionType()} option with "
               f"{payoff_type_to_string_py(payoff)} payoff:\n"
               f"    spot value:       {s}\n"
               f"    strike:           {payoff.strike()}\n"
               f"    dividend yield:   {q_float:.6f}\n"
               f"    risk-free rate:   {r_float:.6f}\n"
               f"    reference date:   {today}\n"
               f"    maturity:         {exercise.lastDate()}\n"
               f"    volatility:       {v_float:.6f}\n\n"
               f"    expected   {greek_name}: {expected}\n"
               f"    calculated {greek_name}: {calculated}\n"
               f"    error:            {error}\n"
               f"    tolerance:        {tolerance}")
        self.fail(msg)

    def test_european_values_no_dividends(self): # Renamed from testEuropeanValues
        print("Testing dividend European option values with no dividends...")
        tolerance = 1.0e-5
        types = [ql.Option.Call, ql.Option.Put]
        strikes = [50.0, 99.5, 100.0, 100.5, 150.0]
        underlyings = [100.0]
        q_rates = [0.00, 0.10, 0.30]
        r_rates = [0.01, 0.05, 0.15]
        lengths = [1, 2] # Years
        vols = [0.05, 0.20, 0.70]

        # Use a fixed eval date for the test run
        current_eval_date = ql.Date(10, ql.July, 2020) # Example fixed date
        ql.Settings.instance().evaluationDate = current_eval_date

        for opt_type in types:
            for strike in strikes:
                for length in lengths:
                    ex_date = current_eval_date + ql.Period(length, ql.Years)
                    exercise = ql.EuropeanExercise(ex_date)

                    # Zero dividends for this test part
                    dividend_dates = ql.DateVector()
                    dividend_amounts = ql.DoubleVector()
                    # DividendVector for AnalyticDividendEuropeanEngine
                    dividend_vector = ql.DividendVector(dividend_dates, dividend_amounts)

                    # DividendVector for the second engine (dividend after expiry)
                    div_date_after_expiry = ex_date + ql.Period(6, ql.Months)
                    dividend_vector2 = ql.DividendVector([div_date_after_expiry], [1.0])


                    payoff = ql.PlainVanillaPayoff(opt_type, strike)
                    ref_engine = ql.AnalyticEuropeanEngine(self.process)
                    ref_option = ql.VanillaOption(payoff, exercise)
                    ref_option.setPricingEngine(ref_engine)

                    engine1 = ql.AnalyticDividendEuropeanEngine(self.process, dividend_vector)
                    option1 = ql.VanillaOption(payoff, exercise)
                    option1.setPricingEngine(engine1)

                    engine2 = ql.AnalyticDividendEuropeanEngine(self.process, dividend_vector2)
                    option2 = ql.VanillaOption(payoff, exercise)
                    option2.setPricingEngine(engine2)

                    for u_val in underlyings:
                        for q_val in q_rates:
                            for r_val in r_rates:
                                for v_val in vols:
                                    self._update_market_data(current_eval_date, u_val, q_val, r_val, v_val)

                                    expected = ref_option.NPV()
                                    calculated1 = option1.NPV()
                                    calculated2 = option2.NPV()
                                    error1 = abs(calculated1 - expected)
                                    error2 = abs(calculated2 - expected)

                                    if error1 > tolerance:
                                        self._report_failure("value (zero divs)", payoff, exercise, u_val, q_val, r_val,
                                                             current_eval_date, v_val, expected, calculated1, error1, tolerance)
                                    if error2 > tolerance:
                                        self._report_failure("value (div after expiry)", payoff, exercise, u_val, q_val, r_val,
                                                             current_eval_date, v_val, expected, calculated2, error2, tolerance)

    def test_european_known_value(self):
        print("Testing dividend European option against known value (Hull)...")
        tolerance = 1.0e-2
        expected = 3.67
        current_eval_date = ql.Date(20, ql.November, 2005) # Example fixed date
        ql.Settings.instance().evaluationDate = current_eval_date

        self._update_market_data(current_eval_date, 40.0, 0.0, 0.09, 0.30) # s, q, r, v

        ex_date = current_eval_date + ql.Period(180, ql.Days)
        exercise = ql.EuropeanExercise(ex_date)

        dividend_dates_list = [current_eval_date + ql.Period(2*30, ql.Days),
                               current_eval_date + ql.Period(5*30, ql.Days)]
        dividend_amounts_list = [0.50, 0.50]
        dividend_vector = ql.DividendVector(ql.DateVector(dividend_dates_list),
                                            ql.DoubleVector(dividend_amounts_list))

        payoff = ql.PlainVanillaPayoff(ql.Option.Call, 40.0)
        engine = ql.AnalyticDividendEuropeanEngine(self.process, dividend_vector)
        option = ql.VanillaOption(payoff, exercise)
        option.setPricingEngine(engine)

        calculated = option.NPV()
        error = abs(calculated - expected)
        if error > tolerance:
            self._report_failure("value (Hull)", payoff, exercise,
                                 self.spot_q.value(), self.q_rate_q.value(), self.r_rate_q.value(),
                                 current_eval_date, self.vol_q.value(),
                                 expected, calculated, error, tolerance)

    # ... Other tests like EuropeanStartLimit, EuropeanGreeks, FdEuropeanValues, FdGreeks, FdDegenerate, FdDividendAtTZero ...
    # These will follow a similar pattern:
    # 1. Set up test parameters (types, strikes, market data ranges, dividend schedules).
    # 2. Set the evaluation date.
    # 3. Loop through parameters.
    # 4. Inside the loop:
    #    a. Call `_update_market_data`.
    #    b. Construct `Exercise`, `Payoff`, `DividendVector`.
    #    c. Construct `VanillaOption` and the appropriate `PricingEngine`.
    #    d. Calculate NPV and/or greeks.
    #    e. Compare with expected values (either analytical, finite difference, or from another engine).
    #    f. Use `_report_failure` or `self.assertAlmostEqual`.

    def test_fd_european_greeks(self):
        print("Testing finite-differences dividend European option greeks...")
        # This is a "Fast" test in C++, so might be computationally intensive
        # We'll test one case to show the pattern.

        current_eval_date = ql.Date(10, ql.August, 2021) # Example fixed date
        ql.Settings.instance().evaluationDate = current_eval_date

        lengths = [1] # Years, use 1 year for brevity

        for length in lengths:
            ex_date = current_eval_date + ql.Period(length, ql.Years)
            exercise = ql.EuropeanExercise(ex_date)
            # Call the private helper that implements the greek testing logic
            self._test_fd_greeks_runner(current_eval_date, exercise, ql.FdBlackScholesVanillaEngine.Spot)
            self._test_fd_greeks_runner(current_eval_date, exercise, ql.FdBlackScholesVanillaEngine.Escrowed)

    def _test_fd_greeks_runner(self, today_for_test, exercise, dividend_model):
        # Based on C++ testFdGreeks (the helper function)
        print(f"  Running FD Greeks for {exercise_type_to_string_py(exercise)}, Model: {dividend_model}")
        tolerances = {"delta": 5.0e-3, "gamma": 7.0e-3} # Add others if needed

        types = [ql.Option.Call, ql.Option.Put]
        strikes = [100.0] # Subset
        underlyings = [100.0]
        q_rates = [0.05] # Subset
        r_rates = [0.05] # Subset
        vols = [0.20]    # Subset

        dc_fd_greeks = ql.Actual365Fixed() # Matches C++ helper

        # Create distinct quotes for FD greek calculation to avoid interference
        spot_q_fd = ql.SimpleQuote(0.0)
        q_rate_q_fd = ql.SimpleQuote(0.0)
        r_rate_q_fd = ql.SimpleQuote(0.0)
        vol_q_fd = ql.SimpleQuote(0.0)

        spot_h_fd = ql.QuoteHandle(spot_q_fd)
        qTS_h_fd = ql.YieldTermStructureHandle(flat_rate_py(today_for_test, q_rate_q_fd, dc_fd_greeks))
        rTS_h_fd = ql.YieldTermStructureHandle(flat_rate_py(today_for_test, r_rate_q_fd, dc_fd_greeks))
        volTS_h_fd = ql.BlackVolTermStructureHandle(flat_vol_py(today_for_test, vol_q_fd, dc_fd_greeks))
        process_fd = ql.BlackScholesMertonProcess(spot_h_fd, qTS_h_fd, rTS_h_fd, volTS_h_fd)

        for opt_type in types:
            for strike in strikes:
                dividend_dates_list = []
                dividend_amounts_list = []
                d_loop = today_for_test + ql.Period(3, ql.Months)
                while d_loop < exercise.lastDate():
                    dividend_dates_list.append(d_loop)
                    dividend_amounts_list.append(5.0)
                    d_loop += ql.Period(6, ql.Months)

                dividend_vector_fd = ql.DividendVector(ql.DateVector(dividend_dates_list),
                                                       ql.DoubleVector(dividend_amounts_list))

                payoff = ql.PlainVanillaPayoff(opt_type, strike)

                # Construct FdBlackScholesVanillaEngine
                # C++ MakeFdBlackScholesVanillaEngine(...).withCashDividends(...).withCashDividendModel(...)
                # Python:
                engine_fd = ql.FdBlackScholesVanillaEngine(process_fd) # Default grids
                engine_fd.setCASH préférenceDividends(dividend_vector_fd) # Typo in example, should be setCashDividends
                engine_fd.setCashDividendModel(dividend_model)
                # Or if newer QL:
                # engine_fd = ql.FdBlackScholesVanillaEngine(process_fd,
                #                                           cashDividends=dividend_vector_fd,
                #                                           cashDividendModel=dividend_model)
                # Assuming setCashDividends and setCashDividendModel exist:
                try: # Try newer syntax first
                    engine_fd_new = ql.FdBlackScholesVanillaEngine(process_fd,
                                                                cashDividends=dividend_vector_fd,
                                                                cashDividendModel=dividend_model)
                    option = ql.VanillaOption(payoff, exercise)
                    option.setPricingEngine(engine_fd_new)
                except TypeError: # Fallback to older setter methods
                    engine_fd_old = ql.FdBlackScholesVanillaEngine(process_fd)
                    engine_fd_old.setCashDividends(dividend_vector_fd)
                    engine_fd_old.setCashDividendModel(dividend_model)
                    option = ql.VanillaOption(payoff, exercise)
                    option.setPricingEngine(engine_fd_old)


                for u_val in underlyings:
                    for q_val in q_rates:
                        for r_val in r_rates:
                            for v_val in vols:
                                spot_q_fd.setValue(u_val)
                                q_rate_q_fd.setValue(q_val)
                                r_rate_q_fd.setValue(r_val)
                                vol_q_fd.setValue(v_val)

                                value = option.NPV()
                                calculated = {"delta": option.delta(), "gamma": option.gamma()}
                                expected = {}

                                if value > spot_q_fd.value() * 1.0e-5:
                                    du = u_val * 1.0e-4
                                    spot_q_fd.setValue(u_val + du)
                                    value_p_spot = option.NPV()
                                    delta_p_spot = option.delta()
                                    spot_q_fd.setValue(u_val - du)
                                    value_m_spot = option.NPV()
                                    delta_m_spot = option.delta()
                                    spot_q_fd.setValue(u_val) # Reset

                                    expected["delta"] = (value_p_spot - value_m_spot) / (2 * du)
                                    expected["gamma"] = (delta_p_spot - delta_m_spot) / (2 * du)

                                    for greek_name, calc_val in calculated.items():
                                        exp_val = expected.get(greek_name)
                                        if exp_val is not None:
                                            tol = tolerances[greek_name]
                                            error = relative_error_py(exp_val, calc_val, u_val)
                                            if error > tol:
                                                self._report_failure(greek_name, payoff, exercise,
                                                                     u_val, q_val, r_val, today_for_test,
                                                                     v_val, exp_val, calc_val, error, tol)
        ql.Settings.instance().evaluationDate = self.saved_eval_date # Restore eval date used by setUp

if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)